Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [297]:
from random import random, seed
from itertools import product
import numpy as np

from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [298]:
UNIVERSE_SIZE = 10000
NUM_SETS = 1000
DENSITY = 0.2

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [299]:
# DON'T EDIT THESE LINES!
SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)


## Helper Functions

In [300]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

## Have Fun!

In [301]:
# A dumb solution of "all" sets
solution = np.full(NUM_SETS, True)
valid(solution), cost(solution)

(np.True_, np.float64(4273189.009129035))

In [302]:
# A random solution with random 50% of the sets
solution = np.random.random(NUM_SETS) < .7
valid(solution), cost(solution)

(np.True_, np.float64(2919009.606331308))

In [303]:
def tweak(old_solution): #simple random tweak
    new_solution=old_solution.copy()
    i=rng.integers(0, NUM_SETS)
    new_solution[i]= not new_solution[i]
    return new_solution

def fitness(solution):
    return (valid(solution),-cost(solution))


In [305]:
def best_neighbor(solution,num_neighbors): # helper to find the best neighbor out of num, using simple tweak
    best=tweak(solution)
    for i in range(num_neighbors-1):
        neighbor=tweak(solution)
        if fitness(neighbor) > fitness(best):
            best=neighbor
    return best

In [310]:
num_of_restarts = 20
steps = 1000
num_of_neighbors = 10
final_solution = rng.random(NUM_SETS) < 1 #all of the sets solution

for i in range(num_of_restarts):
    tmp_solution = rng.random(NUM_SETS) < 0.05 # initialize random solution

    for j in range(steps): #in each step find the best neighbor and check if it's better than current solution
        best_of_neighbor = best_neighbor(tmp_solution,num_of_neighbors)
        if (fitness(best_of_neighbor)>fitness(tmp_solution)):
            tmp_solution = best_of_neighbor

    if(fitness(tmp_solution)>fitness(final_solution)): #check if the tmp solution better than current best
        final_solution = tmp_solution
        ic(fitness(final_solution)) #print new current solution

ic(fitness(final_solution)) # print the final solution
 

ic| fitness(tmp_solution): (np.True_, np.float64(-132305.72410748983))
ic| fitness(final_solution): (np.True_, np.float64(-132305.72410748983))
ic| fitness(tmp_solution): (np.True_, np.float64(-128442.4557832992))
ic| fitness(final_solution): (np.True_, np.float64(-128442.4557832992))
ic| fitness(tmp_solution): (np.True_, np.float64(-128853.3348036522))
ic| fitness(tmp_solution): (np.True_, np.float64(-123335.21406262068))
ic| fitness(final_solution): (np.True_, np.float64(-123335.21406262068))
ic| fitness(tmp_solution): (np.True_, np.float64(-127800.32851672624))
ic| fitness(tmp_solution): (np.True_, np.float64(-124043.17415665704))
ic| fitness(tmp_solution): (np.True_, np.float64(-132434.54917919103))
ic| fitness(tmp_solution): (np.True_, np.float64(-127813.87955245467))
ic| fitness(tmp_solution): (np.True_, np.float64(-132154.94720589346))
ic| fitness(tmp_solution): (np.False_, np.float64(-0.0))
ic| fitness(tmp_solution): (np.True_, np.float64(-123797.90379663011))
ic| fitness(tmp_s

(np.True_, np.float64(-123335.21406262068))